In [3]:
# Import all required libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import plotly.express as px
from animal_shelter import AnimalShelter

# Connect to MongoDB through my CRUD Python module
username = "aacuser"
password = "pass"

# Instantiate AnimalShelter class
db = AnimalShelter(username, password)

# Read all animal data initially
df = pd.DataFrame.from_records(db.read({}))

# Drop Mongo _id field to avoid issues with DataTable
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Encode logo image file
image_filename = 'grazioso_logo.png'  # Ensure this file is in your working directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Start the Dash app
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height': '80px'}
    ),
    html.H1("CS-340 Dashboard - Javaney Thomas", style={'textAlign': 'center'}),
    html.Hr(),

    # Dropdown filter for rescue type
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
            {'label': 'Disaster/Individual Tracking', 'value': 'Disaster/Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        placeholder="Select Rescue Type"
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data={'whiteSpace': 'normal'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
    ),

    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback to filter data based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        breeds = ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]
        query = {"breed": {"$in": breeds}, "age_upon_outcome_in_weeks": {"$lte": 104}}
    elif filter_type == 'Mountain/Wilderness Rescue':
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                  "Siberian Husky", "Rottweiler"]
        query = {"breed": {"$in": breeds}, "age_upon_outcome_in_weeks": {"$lte": 104}}
    elif filter_type == 'Disaster/Individual Tracking':
        breeds = ["German Shepherd", "Doberman Pinscher", "Golden Retriever",
                  "Bloodhound", "Rottweiler"]
        query = {"breed": {"$in": breeds}, "age_upon_outcome_in_weeks": {"$lte": 104}}
    else:
        query = {}

    new_df = pd.DataFrame.from_records(db.read(query))

    if not new_df.empty and '_id' in new_df.columns:
        new_df.drop(columns=['_id'], inplace=True)

    return new_df.to_dict('records')

# Callback for updating breed distribution pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.P("No data to display.")

    dff = pd.DataFrame.from_dict(viewData)

    fig = px.pie(dff, names='breed', title='Breed Distribution')

    return [
        dcc.Graph(figure=fig)
    ]

# Highlight selected columns in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to display location on map for selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(viewData) == 0:
        return html.P("No map data to display.")

    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']
    breed = dff.iloc[row]['breed']
    name = dff.iloc[row]['name']

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[lat, lon],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(name)
                        ])
                    ]
                )
            ]
        )
    ]

# Run the app
app.run_server(debug=True)



Dash app running on http://127.0.0.1:9833/
